In [1]:
import sys
sys.path.append("..")

In [2]:
from src.datasets import HateXplainRace

In [3]:
hate = HateXplainRace('../data/HateXplain')

In [4]:
import faiss
from sentence_transformers import SentenceTransformer
def filter_demographics(
        demographics, overall_demographics
    ) -> str:

        set_of_demographics = set(demographics)

        intersection = set_of_demographics.intersection(overall_demographics)

        if len(intersection) == 0:
            return ""

        else:
            return list(intersection)[0]

embedding = SentenceTransformer('all-mpnet-base-v2')
hate = HateXplainRace("../data/HateXplain/")
train_df, test_df, overall_demographics = hate.create_prompts()
train_df["filtered_demographics"] = train_df["demographics"].apply(
    lambda x: filter_demographics(x, set(hate.demographics))
)

test_df["filtered_demographics"] = test_df["demographics"].apply(
    lambda x: filter_demographics(x, set(hate.demographics))
)

filtered_train_df = train_df[train_df.filtered_demographics != ""].copy()

test_df = test_df[test_df.filtered_demographics != ""]
train_vectors = embedding.encode(train_df['prompts'].tolist(), batch_size = 32, show_progress_bar=True)
test_vectors = embedding.encode(test_df['prompts'].tolist(), batch_size = 32, show_progress_bar=True)
faiss.normalize_L2(train_vectors)

faiss.normalize_L2(test_vectors)
vector_dim = train_vectors.shape[1]

index = faiss.IndexFlatIP(vector_dim)

index.add(train_vectors)

distances, neighbors = index.search(test_vectors, 10)

/home/ksasse/miniconda3/envs/fairness/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Batches: 100%|██████████| 22/22 [00:00<00:00, 41.53it/s]


In [5]:
test_df_index = test_df.reset_index()

In [6]:
differences_in_demographic = []

for i in range(len(neighbors)):
    demographics_train = train_df.iloc[neighbors[i]]['filtered_demographics']
    demographics_test = test_df_index.iloc[i]['filtered_demographics']
    differences_in_demographic.append(len(demographics_train[demographics_train != demographics_test]))
    


In [7]:
import numpy as np

In [8]:
np.array(differences_in_demographic).mean()

3.303964757709251